# Advanced Estimators, Experiments & More

### Loading Libraries

In [1]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# IterTools
from itertools import combinations

# Statistical Models
from scipy import stats
import statsmodels.api as sm

# Networkx
import networkx as nx

# JavaScript Object Notation
import json

# Data Visualization
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt

# Scikit-Learn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV
from sklearn.metrics import mean_absolute_percentage_error

# DoWhy Causal Libraries
import dowhy
from dowhy import gcm
from dowhy import CausalModel
from dowhy.causal_model import CausalModel

# Notebook Iteration 
from tqdm import tqdm

# Copy
from copy import deepcopy

# Light Gradient Boosting Models
from lightgbm import LGBMRegressor, LGBMClassifier

In [ ]:
dowhy.__version__

In [ ]:
COLORS = [
    '#00B0F0',
    '#FF0000',
    '#B0F000'
]

### Doubly Robust Methods: Let’s Get More!

In [ ]:
def plot_effect(effect_true, effect_pred, figsize=(10, 7), ylim=(5000, 22000)):
    plt.figure(figsize=figsize)
    plt.scatter(effect_true, effect_pred, color=COLORS[0])
    plt.plot(np.sort(effect_true), np.sort(effect_true), color=COLORS[1], alpha=.7, label='Perfect model')
    plt.xlabel('$True\ effect$', fontsize=14, alpha=.5)
    plt.ylabel('$Predicted\ effect$', fontsize=14, alpha=.5)
    plt.ylim(ylim[0], ylim[1])
    plt.legend()
    plt.show()

#### Reading Data

In [ ]:
# Train set
earnings_interaction_train = pd.read_csv('/Users/isisromero/desktop/causal_inference/data/ml_earnings_interaction_train.csv')

# Test set
earnings_interaction_test = pd.read_csv('/Users/isisromero/desktop/causal_inference/data/ml_earnings_interaction_test.csv')

In [ ]:
earnings_interaction_train.shape, earnings_interaction_test.shape

In [ ]:
# Train 
earnings_interaction_train.head()

In [ ]:
# Test
earnings_interaction_test.head()

#### Setting Graph

In [ ]:
# Construct the graph (the graph is constant for all iterations)
nodes = ['took_a_course', 'python_proficiency', 'earnings', 'age']
edges = [
    ('took_a_course', 'earnings'),
    ('age', 'took_a_course'),
    ('age', 'earnings'),
    ('python_proficiency', 'earnings')
]

# Generate the GML graph
gml_string = 'graph [directed 1\n'

for node in nodes:
    gml_string += f'\tnode [id "{node}" label "{node}"]\n'

for edge in edges:
    gml_string += f'\tedge [source "{edge[0]}" target "{edge[1]}"]\n'
    
gml_string += ']'

In [ ]:
# Instantiate the CausalModel 
model = CausalModel(
    data=earnings_interaction_train,
    treatment='took_a_course',
    outcome='earnings',
    effect_modifiers='python_proficiency',
    graph=gml_string
)

In [ ]:
model.view_model()

#### Getting The Estimand

In [ ]:
# Get the estimand
estimand = model.identify_effect()

print(estimand)

#### Estimate The Effect

In [ ]:
# Get estimate (Doubly robust)
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dr.LinearDRLearner',
    target_units='ate',
    method_params={
        'init_params': {
            'model_propensity': LogisticRegression(),
            'model_regression': LGBMRegressor(n_estimators=1000, max_depth=10)
        },
        'fit_params': {}
    })

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.causal_estimator.effect(earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1))

# Get the true effect
effect_true = earnings_interaction_test['true_effect'].values

In [ ]:
# Compute the error 
mean_absolute_percentage_error(effect_true, effect_pred)

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### Non-Linear DR

In [ ]:
# Get estimate (Doubly robust)
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dr.DRLearner',
    target_units='ate',
    method_params={
        'init_params': {
            'model_propensity': LogisticRegression(),
            'model_regression': LGBMRegressor(n_estimators=1000, max_depth=10),
            'model_final': LGBMRegressor(n_estimators=500, max_depth=10),
        },
        'fit_params': {}
    })# Compute predictions
effect_pred = model.causal_estimator.effect(earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1))

# Get the true effect
effect_true = earnings_interaction_test['true_effect'].values

In [ ]:
# Compute predictions
effect_pred = model.causal_estimator.effect(earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1))

# Get the true effect
effect_true = earnings_interaction_test['true_effect'].values

In [ ]:
# Compute the error 
mean_absolute_percentage_error(effect_true, effect_pred)

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### If Machine Learning is Cool, How About Double Machine Learning?

#### Estimating The Effect

In [ ]:
# Get estimate (DML)
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dml.LinearDML',
    target_units='ate',
    method_params={
        'init_params': {
            'model_y': LGBMRegressor(n_estimators=500, max_depth=10),
            'model_t': LogisticRegression(),
            'discrete_treatment': True
        },
        'fit_params': {}
    })

In [ ]:
estimate.cate_estimates.mean()

#### Predicting on Test Data

In [ ]:
# Compute predictions
effect_pred = model.causal_estimator.effect(earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1))

# Get the true effect
effect_true = earnings_interaction_test['true_effect'].values

In [ ]:
# Compute the error 
mean_absolute_percentage_error(effect_true, effect_pred)

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### DML with more Folds

In [ ]:
# Estimate the effect
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dml.LinearDML',
    target_units='ate',
    method_params={
        'init_params': {
            'model_y': LGBMRegressor(n_estimators=50, max_depth=10),
            'model_t': LogisticRegression(),
            'discrete_treatment': True,
            'cv': 4
        },
        'fit_params': {
        }
    })

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.causal_estimator.effect(earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1))

# Get the true effect
effect_true = earnings_interaction_test['true_effect'].values

In [ ]:
# Compute the error 
mean_absolute_percentage_error(effect_true, effect_pred)

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

#### DML with Cross-Validation

In [ ]:
# Define wrapped CV models
model_y = GridSearchCV(
    estimator=LGBMRegressor(),
    param_grid={
        'max_depth': [3, 10, 20, 100],
        'n_estimators': [10, 50, 100]
    }, cv=10, n_jobs=-1, scoring='neg_mean_squared_error'
)

model_t = GridSearchCV(
    estimator=LGBMClassifier(),
    param_grid={
        'max_depth': [3, 10, 20, 100],
        'n_estimators': [10, 50, 100]
    }, cv=10, n_jobs=-1, scoring='accuracy'
)

In [ ]:
# Estimate the effect
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dml.LinearDML',
    target_units='ate',
    method_params={
        'init_params': {
            'model_y': model_y,
            'model_t': model_t,
            'discrete_treatment': True,
            'cv': 4
        },
        'fit_params': {
        }
    })

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.causal_estimator.effect(earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1))

# Get the true effect
effect_true = earnings_interaction_test['true_effect'].values

In [ ]:
# Compute the error 
mean_absolute_percentage_error(effect_true, effect_pred)

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### Causal Forests & More

In [ ]:
# Estimate the effect
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dml.CausalForestDML',
    target_units='ate',
    method_params={
        'init_params': {
            'model_y': LGBMRegressor(n_estimators=50, max_depth=10),
            'model_t': LGBMClassifier(n_estimators=50, max_depth=10),
            'discrete_treatment': True,
            'cv': 4
        },
        'fit_params': {
        }
    }
)

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.causal_estimator.effect(earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1))

# Get the true effect
effect_true = earnings_interaction_test['true_effect'].values

In [ ]:
# Compute the error 
mean_absolute_percentage_error(effect_true, effect_pred)

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### Heterogenous Treatment Effects With Experimental Data

In [ ]:
# Read in the data
hillstrom_clean = pd.read_csv(r'./data/hillstrom_clean.csv')

# Read in labels mapping
with open(r'./data/hillstrom_clean_label_mapping.json', 'r') as f:
    hillstrom_labels_mapping = json.load(f)

In [ ]:
hillstrom_clean.head()

In [ ]:
# Drop redundant cols to avoid multicollinearity
hillstrom_clean = hillstrom_clean.drop(['zip_code__urban', 'channel__web'], axis=1)

#### EDA

In [ ]:
# Display mapping
hillstrom_labels_mapping

In [ ]:
# P(visit)
hillstrom_clean.visit.mean()

In [ ]:
# P(conversion)
hillstrom_clean.conversion.mean()

In [ ]:
# Get sample size
sample_size = hillstrom_clean.shape[0]

In [ ]:
# Check how random is the treatment assignment

# Split data
hillstrom_X = hillstrom_clean.drop(['visit', 'conversion', 'spend', 'treatment'], axis=1)
hillstrom_Y = hillstrom_clean['spend']
hillstrom_T = hillstrom_clean['treatment']

In [ ]:
# P(T=t)
hillstrom_T.value_counts() / sample_size

In [ ]:
# Train-test split
X_train_eda, X_test_eda, T_train_eda, T_test_eda = train_test_split(hillstrom_X, hillstrom_T, test_size=.5)

In [ ]:
# Split quality
T_test_eda.value_counts() / T_test_eda.shape[0]

In [ ]:
# Fit the EDA model
lgbm_eda = LGBMClassifier()
lgbm_eda.fit(X_train_eda, T_train_eda)

In [ ]:
# Get predictions on the test
T_pred_eda = lgbm_eda.predict(X_test_eda)

# Check accuracy
acc_eda = accuracy_score(T_test_eda, T_pred_eda)
acc_eda

In [ ]:
# Generate random data
random_scores = []

test_eda_sample_size = T_test_eda.shape[0]

for i in range(10000):
    random_scores.append(
        (np.random.choice(
            [0, 1, 2], 
            test_eda_sample_size) == np.random.choice(
            [0, 1, 2], 
            test_eda_sample_size)).mean())
    
np.quantile(random_scores, .025), np.quantile(random_scores, .975)

In [ ]:
# Get 95% CIs
lower = np.quantile(random_scores, .025)
upper = np.quantile(random_scores, .975)
lower, upper

In [ ]:
# Plot radom vs accuracy
plt.figure(figsize=(10, 6))
plt.fill_betweenx(
    x1=lower, 
    x2=upper, 
    y=np.arange(0, 300),
    color=COLORS[0],
    alpha=.1,
    label='95% empirical CI'
)
plt.hist(random_scores, alpha=.7, color=COLORS[0], bins=100, label='Random models\n($n=10e3$)')
plt.axvline(acc_eda, color=COLORS[1], ls='--', label='Model accuracy')

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

plt.hist(
    hillstrom_clean[hillstrom_clean['treatment'] == 2]['spend'], 
    label=f'Men\'s', 
    color=COLORS[0],
    bins=100, 
    alpha=.5)

plt.hist(
    hillstrom_clean[hillstrom_clean['treatment'] == 1]['spend'], 
    label=f'Women\'s',
    color=COLORS[1],
    bins=100, 
    alpha=.5)

plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
hillstrom_clean[hillstrom_clean['treatment'] == 1]['spend'].var()

In [ ]:
hillstrom_labels_mapping

### Modeling Hillstrom

In [ ]:
# No. of conversions
(hillstrom_Y[hillstrom_T > 0] > 0).sum()